## <p style="text-align: center;">MIS 284N - Big Data and Distributed Programming</p>
## <p style="text-align: center;">Project 3 - Machine Learning using Tensorflow and Google Colab</p>
## <p style="text-align: center;">Total points: 100</p>
## <p style="text-align: center;">Due: Sunday, October 20th submitted via Canvas by 11:59 pm</p>

Your homework should be written in a **Jupyter notebook**. You may work in groups of two if you wish. Only one student per team needs to submit the assignment on Canvas.  But be sure to include name and UTID for both students.

Also, please make sure your code runs and the graphics (and anything else) are displayed in your notebook before submitting. (%matplotlib inline)

This project is about giving exposure about Tensorflow, its usage, Cloud services and help us in understanding the time taken to run computation on CPU and GPU. 

In this Project, we will work with CIFAR10 image dataset. 
The starter code to download the dataste using keras is given below. 
You should run this project on Google Colab. You would be using CPU, GPU.
Use tensorflow version 2.0. 

# In every line of code, please write a comment to briefly explain what that line is doing.
Your grades will be based on your understanding of the code you write! 

Note: The code you write should be your own!

# Task 1
Convert the features in a form that can be given as input to tensorflow library/functions

In this task you will perform data augmentation. That is, pre-process the data to make the model more robust. Most common data augmentation techniques are rotation, flips and histogram equalization. 
You can choose an augmentation technique of your choice. 

In [2]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [4]:
import time
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [7]:
from keras.datasets import cifar10

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

datagen = ImageDataGenerator(width_shift_range=0.1, horizontal_flip=True)
datagen.fit(X_train)

# Task 2
Try to build a Neural Network model, train on the features and report the accuracy.
Report your observations on the time taken on GPU and TPUs. 



1.   Create a CNN based model with 5 hidden layers and 100 hidden units each layer. 
2.   Create an LSTM based model with 2 hidden layers and 1024 hidden units in each layer. 



In [1]:
from keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

Using TensorFlow backend.


170500096/170498071 [==============================] - 11s 0us/step


# Task 3 (Extra credit, 25 points)
Run the above on a TPU and report the time taken to fit the models. 